# 1. Import the required libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

# 2. Load the data

## Loading the forest inventory table
Load the `forest inventory` table (from *PostGIS*, *JSON* or *shapefile*) into a `geopandas` dataframe and observe the data

In [ ]:
gdf = gpd.read_file('./shapes/shapes.shp')
gdf

## Loading the plot information data (collected from the projects)
Static information about the projects (a.k.a. `plot information`) may be collected from (i) Conversations with project stakeholders, (ii) Survey forms (or) (iii) Project design documentation. Load this data to check if it is complete or if any data needs to be corrected or imputed

In [ ]:
df = pd.read_csv('./data/Plot information.csv')
df

## Insert the obtained plot information data into the PostGIS database

Per our architecture, static information about the projects (a.k.a. `plot information`) once collected, is saved in the `T2 - Plot information` table. Insert the data into the PostGIS table using a SQL statement

In [ ]:
#INSERT INTO SQL STATEMENT

# 2. Merge the data

## (use a horizontal concatenation operation, using the axis parameter)
If both the `forest inventory` table and the `plot information` data are up-to-date and clean, merge the two dataframes and reset the index information. Drop any duplicated columns (such as `Plot ID` i.e. `PID`)

In [ ]:
df.rename(columns={"Plot ID":"PID"}, inplace=True)
gdf = pd.concat([gdf, df], axis=1)
gdf.drop(columns=gdf.columns[3],inplace=True)
gdf['PID'] = ['10000', '10001', '10002', '10003', '10004', '10005']
gdf.set_index("PID", inplace=True)
gdf

# 3. Save the data

## With polygonal geometry
Typically vector information involving areas (states, counties, regions etc..) are plotted as vector polygons with attributes in readable widgets or popups. Save the joined vector table as (i) `ESRI shapefile` (ii) `Geojson table`

In [ ]:
gdf.to_file("./shapes/plot_info.shp")
gdf.to_file("./shapes/geojson/plot_info.geojson", driver='GeoJSON')

## With point geometry
One may be interested to plot the vector data, not as polygons with attributes, but rather as points (i.e. marker clusters) with popups. This means we would need to make a discretion around which point to consider within a polygon. The most common and appropriate choice is the `centroid` of the polygon.

In [ ]:
#computing polygon centroids using a built in geopandas method
gdf['centroid'] = gdf.centroid
gdf

Drop the geometry column, rename the centroid column a geometry and save the table with the point geometry as (i) `ESRI shapefile` (ii) `Geojson table`

In [ ]:
gdf.drop(columns="geometry", inplace=True)
gdf.rename(columns={"centroid": "geometry"}, inplace=True)
gdf.to_file("./shapes/plot_info_centroids.shp")
gdf.to_file("./shapes/geojson/plot_info_centroids.geojson", driver='GeoJSON')

# 3. Visualize the vector data to check accuracy

In order to ensure that the saved vector tables (with `polygonal` and `point` geometry) are accurate, 
(i) create a new interactive map with Geemap, and use the UI controls to load the shapefile / geojson tables
(ii) use the UI controls to alter the colormap and other presentation elements as required

In [ ]:
#Point cluster plot (centroids)
Map = geemap.Map()
Map

---
Copyright © 2022 Carbon Capital Advisors. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.